In [ ]:
import numpy as np
import cv2
from detection import detect_objects
import os
import copy
import json
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
def select_frames(video_path, dst_path, n_frames=100):
    # parameters:
    #   video_path - absolute or relative path to video file
    #   dst_path - absolute or relative path to result frames folder
    #   n_frames - number of frames to sample
    cap = cv2.VideoCapture(video_path)
    count = 0
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_list = np.linspace(0, total_frames, n_frames, dtype=int)
    # folder = video_path.split('.')[0]
    frame_count = 0
    print('video fps:', fps)
    print('video total frames:', total_frames)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count in frame_list:
            cv2.imwrite(f'{dst_path}/frame{str(frame_count).zfill(6)}.jpg', frame)
            frame_count += 1
        count += 1
    cap.release()
    cv2.destroyAllWindows()
    print('Frame selection: done')

In [ ]:
thresholds = {
    'bowl': 250,
    'orange': 200,
    'teddybear': 200
}
def get_co3d_masks(mask_path, threshold):
    # parameters:
    #   mask_path - absolute or relative path to mask folder
    for file in tqdm(os.listdir(mask_path), desc='CO3D mask processing..'):
        img = cv2.imread(mask_path + '/' + file)
        img_w = copy.deepcopy(img)
        img_w[img_w >= threshold] = 255
        img_w[img_w < threshold] = 0
        cv2.imwrite(mask_path + '/' + file, img_w)

        
def get_colmap_masks(mask_path):
    # parameters:
    #   mask_path - absolute or relative path to mask folder
    for file in tqdm(os.listdir(mask_path), desc='COLMAP mask processing..'):
        file = os.path.abspath(mask_path + '\\' + file)
        new_file = '\\'.join(file.split('\\')[:-1]) + \
        '\\' + \
        file.split('\\')[-1].replace('mask', 'frame')
        os.rename(file, new_file + '.png')
        
        
def get_masks(frame_path, mask_path, coco_class, frame_process=None, colmap_process=False):
#folder_path = r"C:\Users\marem\PycharmProjects\home\projects\unn\cw_1\frame_selector\frames\chair_auto"
    # parameters:
    #   frame_path - absolute or relative path to frames folder
    #   mask_path - absolute or relative path to mask folder
    #   coco_class - COCO class to recognize
    #   frame_process - what to do with frames on which object was not recognized:
    #     delete - delete such frames
    #     mask - make full frame masks for such frames
    #   colmap_process - whether to process masks for COLMAP format or not
    result = detect_objects(frame_path)
    folder_prefix = frame_path.split('\\')[-1]
    names_list = []
    for file in os.listdir(mask_path):
        file = os.path.abspath(mask_path + file)
        os.remove(file)
    for i, res in enumerate(tqdm(result, desc='Mask processing..')):
        best_obj = None
        best_obj_score = -1
        best_obj_area = -1
        for obj in res[1]:
            obj_coords = np.array(obj[1], dtype='int')
            obj_area = abs(obj_coords[1]-obj_coords[3]) * abs(obj_coords[0]-obj_coords[2])
            if obj[0] == coco_class:
                if obj_area > best_obj_area:
                    best_obj = obj
                    best_obj_area = obj_area

        if best_obj is None:
            continue
        names_list.append(res[0])
        coords = np.array(best_obj[1], dtype='int')
        img = cv2.imread(res[0])
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray[:, :] = 0
        gray[coords[1]:coords[3], coords[0]:coords[2]]=255
        mask_name = res[0].split('\\')[-1].replace('frame', 'mask')
        cv2.imwrite(mask_path + mask_name, gray)
    
    if frame_process == 'delete':
        for file in tqdm(os.listdir(frame_path), desc='Frame deletion..'):
            file = os.path.abspath(frame_path + '/' + file)
            if file not in names_list:
                os.remove(file)
    elif frame_process == 'mask':
        for file in tqdm(os.listdir(frame_path), desc='Frame masking..'):
            file = os.path.abspath(frame_path + '/' + file)
            if file not in names_list:
                img = cv2.imread(file)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray[:, :] = 255
                mask_name = file.split('\\')[-1].replace('frame', 'mask')
                cv2.imwrite(mask_path + mask_name, gray)
    if colmap_process:
        get_colmap_masks(mask_path)
    if co3d_threshold is not None:
        get_co3d_masks(mask_path, co3d_threshold)
    print('Masks processing: done')

In [11]:
import os
import shutil
from tqdm import tqdm

# get OpenMVS formatted masks from COLMAP masks (frame000001.jpg.png -> frame000001.mask.png)
# MASKS IN OPENMVS ARE NOT WORKING AS EXPECTED
def get_openmvs_masks(path_to_colmap_masks, path_to_openmvs_masks):
    for filename in tqdm(os.listdir(path_to_colmap_masks), desc='OpenMVS mask processing..'):
        file_path = os.path.abspath(path_to_colmap_masks + '\\' + filename)
        new_file_path = path_to_openmvs_masks + '\\' + file_path.split('\\')[-1].split('.')[0] + '.mask.png'
        shutil.copyfile(file_path, new_file_path)

get_openmvs_masks(r"<PATH TO MASKS SEGMENT>", r"<PATH TO MASKS OPENMVS>")

OpenMVS mask processing..: 100%|██████████| 100/100 [00:00<00:00, 2088.76it/s]


In [15]:
# get images from all images based on available masks
def get_images_from_masks(path_to_all_images, path_to_masks, path_to_images):
    frame_names = list()
    for filename in tqdm(os.listdir(path_to_masks), desc='Mask filenames storing..'):
        frame_name = filename.split('.')[0]
        frame_names.append(frame_name)
    for filename in tqdm(os.listdir(path_to_all_images), desc='Images copying..'):
        file_path = os.path.abspath(path_to_all_images + '\\' + filename)
        frame_name = filename.split('.')[0]
        if frame_name in frame_names:
            new_file_path = path_to_images + '\\' + filename
            shutil.copyfile(file_path, new_file_path)

get_images_from_masks(
    r"<PATH TO ALL IMAGES>",
    r"<PATH TO MASKS SEGMENT>",
    r"<PATH TO IMAGES>"
)

Mask filenames storing..: 100%|██████████| 100/100 [00:00<00:00, 100031.10it/s]


Images copying..: 100%|██████████| 202/202 [00:00<00:00, 2728.43it/s]
